In [415]:
import numpy as np
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import NonlinearConstraint
from scipy.optimize import LinearConstraint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from cvxopt.modeling import op, variable, dot
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [416]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels



X_all_labels, y_all_labels = load_mnist('C:/Users/RE-Giorgio/Documents/OptimusPrime/Data', kind='train')


indexLabel2 = np.where((y_all_labels==2))
xLabel2 =  X_all_labels[indexLabel2][:1000,:].astype('float64') 
yLabel2 = y_all_labels[indexLabel2][:1000].astype('float64') 

indexLabel4 = np.where((y_all_labels==4))
xLabel4 =  X_all_labels[indexLabel4][:1000,:].astype('float64') 
yLabel4 = y_all_labels[indexLabel4][:1000].astype('float64') 

indexLabel6 = np.where((y_all_labels==6))
xLabel6 =  X_all_labels[indexLabel6][:100,:].astype('float64') 
yLabel6 = y_all_labels[indexLabel6][:100].astype('float64') 

yLabel2[:] = +1
yLabel4[:] = -1

X = np.concatenate([xLabel2, xLabel4])
y = np.concatenate([yLabel2, yLabel4])



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1696995) 

scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [426]:
class Svm:
    
    def __init__(self, X, y, gamma, C):
        
        self.X = X
        self.y = y
        self.alpha = np.random.normal(0,1,(1,self.X.shape[0]))
        self.b = np.random.randint(1)
        self.C = C
        self.gamma = gamma
        
    def predict(self,X):
        
        z = (self.alpha*self.y) @ self.kernel_gauss(self.X, X) + self.b
        a = np.sign(z)    
        return a
        
    def kernel_gauss(self, X1, X2):
        
        K  = np.zeros((X1.shape[0],X2.shape[0]))
        for i in range(X1.shape[0]):
            for j in range(X2.shape[0]):
                K[i,j] = np.exp(-self.gamma * (np.square(np.linalg.norm(X1[i] - X2[j]))))
        return K
        
    def optimize(self):
        
        # computing alpha
        y = self.y.reshape(-1,1)
        K = self.kernel_gauss(self.X, self.X)
        H = np.outer(y,y) * K
        P = cvxopt_matrix(H)
        q = cvxopt_matrix(-np.ones((self.X.shape[0])))
        G = cvxopt_matrix(np.vstack((-np.eye(self.X.shape[0]),np.eye(self.X.shape[0]))))
        h = cvxopt_matrix(np.hstack((np.zeros(self.X.shape[0]), np.ones(self.X.shape[0]) * self.C)))
        A = cvxopt_matrix(y.reshape(1, -1))
        b = cvxopt_matrix(np.zeros(1))
        
        res = cvxopt_solvers.qp(P, q, G, h, A, b)
        
        alpha = np.array(res['x'])
        self.alpha = alpha.T
        
        # computing b
        alpha = alpha.ravel()
        idx = np.where(alpha > 1e-5)[0]
        b = 0
        for i in range(len(idx)):
            b += y[i] - np.sum(alpha[idx] * self.y[idx] * K[idx[i],idx])
        self.b = b/len(idx)
        
svm = Svm(X_train, y_train, gamma = 0.1, C = 1)
svm.optimize()
y_pred = svm.predict(X_test)
print(accuracy_score(y_test.reshape(-1,1), y_pred.reshape(-1,1)))

0.835


In [302]:
from sklearn.svm import SVC
clf = SVC(gamma=0.1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test.reshape(-1,1), y_pred.reshape(-1,1)))

0.8325


In [305]:
clf.

<bound method BaseSVC.decision_function of SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)>